In [27]:
import os
import pandas as pd

from_time = 1800
interval_value = 60

mer_path = r"C:\VISSIM_Workspace\network_new\network_final\network01_mer\network01_001.mer"
with open(mer_path, "r", encoding="utf-8", errors="ignore") as file:
    lines = file.readlines()
# 데이터가 시작하는 인덱스 찾기
data_start_idx = None

for j, line in enumerate(lines):
    if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
        data_start_idx = j
        break
df_list = []
# 데이터프레임 생성
if data_start_idx is not None:

    # 컬럼명 추출
    columns = lines[data_start_idx].strip().split(";")

    # 데이터 부분 추출 및 가공
    data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
    data = [line.strip().split(";") for line in data_lines if line.strip()]

    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=columns)
    # 컬럼명을 정리 (혹시 모를 공백 제거)
    df.columns = df.columns.str.strip()

    # 변환할 숫자형 컬럼 리스트
    numeric_columns = ["Measurem.", "t(Entry)", "t(Exit)", "VehNo", "Vehicle type",
                       "Line", "v[km/h]", "b[m/s2]", "Occ", "Pers", "tQueue", "VehLength[m]"]

    # 각 컬럼을 숫자로 변환
    for col in numeric_columns:
        if col in df.columns:  # 컬럼이 존재하는지 확인
            df[col] = pd.to_numeric(df[col], errors="coerce")  # 변환 실패 시 NaN 처리
    df["time"] = df[["t(Entry)", "t(Exit)"]].max(axis=1)
    # 시간대별 범위 적용
    current_time = from_time
    max_time = df["time"].max()

    while current_time <= max_time:
        next_time = current_time + interval_value
        df.loc[df["time"].between(current_time, next_time, inclusive="right"), "time"] = next_time
        df.loc[df["time"] == 1800, "time"] = 1860
        current_time = next_time
    df = df.groupby(["Vehicle type", "Measurem."])[["time", "b[m/s2]", "Occ", "tQueue", "VehLength[m]", "v[km/h]"]].agg("mean").reset_index().round(2)
    df["time"] = df["time"].astype(int)
    #print(df)
    excel_path = r"C:\VISSIM_Workspace\network_new\network_final\network01_mer\mer_output\aaaaaa.xlsx"
    df.to_excel(excel_path, index=False)